In [1]:
import pyam
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.lines import Line2D
import matplotlib.transforms as mtransforms

from pathlib import Path
import dotenv
import os

<IPython.core.display.Javascript object>

In [2]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [3]:
dotenv.load_dotenv()

True

<IPython.core.display.Javascript object>

Step 1: Read in the necessary data for this assessment. This is basically the metadata that is crunched in the notebook `101_prepare_input_data.ipynb`

In [4]:
df = pd.read_excel(
    Path(
        '../data/101_data.xlsx'
    ),
    sheet_name='meta',
    index_col=[0,1]
)

<IPython.core.display.Javascript object>

In [5]:
delay_categories = pd.read_excel(
    Path(
        '../data/102_delay_categories.xlsx'
    ),
    index_col=[0,1]
)

<IPython.core.display.Javascript object>

Step 2: Let us pull out the necessary columns.

In [6]:
columns_of_interest = [
    'Category',
    'cumulative_gross_netzero_2100',
    'cumulative_cdr_netzero_2100',
]

<IPython.core.display.Javascript object>

In [7]:
df_for_analysis = df[columns_of_interest].dropna()

<IPython.core.display.Javascript object>

Step 3: Calculate the total amount which is used to balance ongoing gross emissions.

In [8]:
df_for_analysis['balance'] = (
    df_for_analysis[['cumulative_gross_netzero_2100', 'cumulative_cdr_netzero_2100']]
    .min(axis=1)
)

<IPython.core.display.Javascript object>

Step 4: Calculate the amount used for decline.

In [9]:
df_for_analysis['decline'] = (
    df_for_analysis['cumulative_cdr_netzero_2100']
    -
    df_for_analysis['balance']
)

<IPython.core.display.Javascript object>

Step 5: Calculate the shares for balance and decline.

In [10]:
df_for_analysis['share_balance'] = (
    df_for_analysis['balance']
    *
    100
    /
    df_for_analysis['cumulative_cdr_netzero_2100']
)

<IPython.core.display.Javascript object>

In [11]:
df_for_analysis['share_decline'] = (
    100
    -
    df_for_analysis['share_balance']
)

<IPython.core.display.Javascript object>

Step 6: Get the summary statistics for the shares as well as the volumes.

In [12]:
summary_share = (
    df_for_analysis
    .groupby('Category')
    [['share_balance', 'share_decline']]
    .describe()
    .round(0)
)

<IPython.core.display.Javascript object>

In [13]:
summary_volume = (
    df_for_analysis
    .groupby('Category')
    [['balance', 'decline']]
    .describe()
    .round(0)
)

<IPython.core.display.Javascript object>

In [14]:
print(
    f"""
    In the assessed C1 scenarios, 
    {summary_share.loc['C1', pd.IndexSlice['share_balance', '50%']]} 
    [{summary_share.loc['C1', pd.IndexSlice['share_balance', '25%']]}, {summary_share.loc['C1', pd.IndexSlice['share_balance', '75%']]}]
    (in volume terms 
    {summary_volume.loc['C1', pd.IndexSlice['balance', '50%']]} 
    [{summary_volume.loc['C1', pd.IndexSlice['balance', '25%']]}, {summary_volume.loc['C1', pd.IndexSlice['balance', '75%']]}]) 
    of the total CDR deployed over this timeframe is to balance residual gross CO2 emissions.
    """
)


    In the assessed C1 scenarios, 
    55.0 
    [44.0, 91.0]
    (in volume terms 
    342.0 
    [245.0, 426.0]) 
    of the total CDR deployed over this timeframe is to balance residual gross CO2 emissions.
    


<IPython.core.display.Javascript object>

In [15]:
print(
    f"""
    The corresponding values for C2 scenarios are somewhat lower at 
    {summary_share.loc['C2', pd.IndexSlice['share_balance', '50%']]} 
    [{summary_share.loc['C2', pd.IndexSlice['share_balance', '25%']]}, {summary_share.loc['C2', pd.IndexSlice['share_balance', '75%']]}]
    (in volume terms 
    {summary_volume.loc['C2', pd.IndexSlice['balance', '50%']]} 
    [{summary_volume.loc['C2', pd.IndexSlice['balance', '25%']]}, {summary_volume.loc['C2', pd.IndexSlice['balance', '75%']]}]) 
    linked to a greater demand for CDR to reverse temperature overshoot in C2 pathways when compared to C1 pathways.
    """
)


    The corresponding values for C2 scenarios are somewhat lower at 
    50.0 
    [40.0, 60.0]
    (in volume terms 
    298.0 
    [236.0, 384.0]) 
    linked to a greater demand for CDR to reverse temperature overshoot in C2 pathways when compared to C1 pathways.
    


<IPython.core.display.Javascript object>

In [16]:
print(
    f"""
    higher volumes of CDR are deployed in C2 pathways to reverse warming
    {summary_volume.loc['C2', pd.IndexSlice['decline', '50%']]} 
    [{summary_volume.loc['C2', pd.IndexSlice['decline', '25%']]}, {summary_volume.loc['C2', pd.IndexSlice['decline', '75%']]}])
    when compared to the C1 pathways
    {summary_volume.loc['C1', pd.IndexSlice['decline', '50%']]} 
    [{summary_volume.loc['C1', pd.IndexSlice['decline', '25%']]}, {summary_volume.loc['C1', pd.IndexSlice['decline', '75%']]}])
    """
)


    higher volumes of CDR are deployed in C2 pathways to reverse warming
    309.0 
    [221.0, 424.0])
    when compared to the C1 pathways
    184.0 
    [26.0, 418.0])
    


<IPython.core.display.Javascript object>

Step 7: Calculate the shares of novel CDR after net zero CO2.

In [17]:
df_cdr = (
    df[['Category', 'cumulative_cdr_netzero_2100','novel_cdr_netzero_2100']]
)

<IPython.core.display.Javascript object>

In [18]:
df_cdr['share_novel'] = (
    df_cdr['novel_cdr_netzero_2100']
    *
    100
    /
    df_cdr['cumulative_cdr_netzero_2100']
)

/var/folders/nz/kr7bwztj0_d9xrfz1jd4zcdh0000gp/T/ipykernel_4086/4132692584.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cdr['share_novel'] = (


<IPython.core.display.Javascript object>

In [19]:
df_cdr['share_novel'][df_cdr['share_novel'] >100] = 100

/var/folders/nz/kr7bwztj0_d9xrfz1jd4zcdh0000gp/T/ipykernel_4086/2032100587.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cdr['share_novel'][df_cdr['share_novel'] >100] = 100


<IPython.core.display.Javascript object>

In [20]:
summary_novel_shares = (
    df_cdr['share_novel']
    .describe()
    .round(0)
)

<IPython.core.display.Javascript object>

In [21]:
summary_novel_shares_cat = (
    df_cdr
    .groupby('Category')['share_novel']
    .describe()
    .round(0)
)

<IPython.core.display.Javascript object>

In [22]:
summary_novel_shares

count    401.0
mean      66.0
std       17.0
min        0.0
25%       58.0
50%       66.0
75%       79.0
max      100.0
Name: share_novel, dtype: float64

<IPython.core.display.Javascript object>

In [23]:
summary_novel_shares_cat

,count,mean,std,min,25%,50%,75%,max
Category,,,,,,,,
C1,70.0,59.0,19.0,0.0,48.0,62.0,72.0,90.0
C2,102.0,70.0,12.0,48.0,60.0,66.0,80.0,100.0
C3,229.0,67.0,17.0,0.0,57.0,68.0,78.0,100.0


<IPython.core.display.Javascript object>